### Task 1. Distinguishing into strata

Suppose we have a set of features that are computed independently of the experiment. \
We need to stratify the objects so that:
- the variance of the stratified mean is minimize
- the variance <= 50_000
- the proportion of each stratum is at least 5% of the total data

In [38]:
import pandas as pd
import numpy as np

def get_strats(df_features):
    """Возвращает страты объектов.

    :param df_features (pd.DataFrame): таблица с признаками x1,...,x10
    :return (list | np.array | pd.Series): список страт объектов размера len(df).
    """
    df = df_features.copy()
    df['x2'] = df['x2'].clip(lower=25,upper=35)
    df['x10'] = df['x10'].clip(upper=5)
    df['strat'] = [' '.join(values) for values in df[['x2', 'x10']].astype(str).values]
    return df['strat']

In [115]:
df = pd.read_csv('stratification_task_data_public.csv')
df_features = df.drop('y', axis=1)
df['strat'] = get_strats(df_features)

In [116]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y,strat
0,0.869,30,33.8,0,1,0.2,1992,1,1,1,1903,30 1
1,0.759,27,21.7,2,0,3.5,1995,1,1,2,1313,27 2
2,0.456,29,37.6,2,0,3.1,1993,0,0,0,1484,29 0
3,0.060,35,27.5,2,0,4.7,1988,0,0,1,1188,35 1
4,0.939,19,30.7,0,0,3.6,2003,1,1,2,842,25 2
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.844,31,31.7,0,0,1.8,1992,1,0,3,1798,31 3
9996,0.342,27,32.5,1,0,1.8,1996,1,0,0,1457,27 0
9997,0.679,28,29.7,1,0,0.8,1994,1,0,2,1477,28 2
9998,0.724,22,38.4,0,0,4.6,2000,1,0,2,1484,25 2


#### Population mean and deviation

In [118]:
mean_y = round(df['y'].mean(),1)
var_y = round(df['y'].var(),1)
print(mean_y, var_y)

1367.9 66078.1


In [42]:
def _calculate_strat_stat(df):
    strat_var = df.groupby('strat')['y'].var()
    weights = df['strat'].value_counts(normalize=True)
    stratified_var = (strat_var*weights).sum()
    min_part = df['strat'].value_counts(normalize=True).min()
    return stratified_var, min_part

#### Strategy 1. Feature like strat

In [24]:
for feature in [f'x{i}' for i in range(1, 11)]:
    df['strat'] = df[feature]
    stratified_var, min_part = calculate_strat_stat(df)
    print(f'{feature:<5} var={stratified_var:0.0f}, min_part={min_part*100:0.2f}%')

x1    var=66234, min_part=0.02%
x2    var=47232, min_part=0.01%
x3    var=64391, min_part=0.01%
x4    var=66054, min_part=32.61%
x5    var=66022, min_part=40.74%
x6    var=65714, min_part=0.01%
x7    var=48055, min_part=0.01%
x8    var=64859, min_part=34.95%
x9    var=66051, min_part=39.70%
x10   var=63756, min_part=0.07%


#### Strategy 2. Union strat

In [61]:
df_agg = df.groupby('x10')[['y']].agg(['count', 'mean'])
df_agg.columns = ['count', 'mean']
df_agg['part'] = (df_agg['count'] / len(df)).round(3)
df_agg['mean'] = df_agg['mean'].round(1)
df_agg

,count,mean,part
x10,,,
0,1362,1311.5,0.136
1,2701,1310.8,0.270
2,2702,1391.6,0.270
3,1825,1411.9,0.182
4,895,1427.0,0.090
5,346,1428.5,0.035
6,132,1425.7,0.013
7,30,1526.9,0.003
8,7,1402.3,0.001


In [79]:
df['x10'] = df['x10'].clip(upper=5)

In [80]:
df_agg = df.groupby('x2')[['y']].agg(['count', 'mean'])
df_agg.columns = ['count', 'mean']
df_agg['part'] = (df_agg['count'] / len(df)).round(3)
df_agg['mean'] = df_agg['mean'].round(1)
df_agg

,count,mean,part
x2,,,
12,2,572.5,0.000
15,2,677.5,0.000
16,8,746.8,0.001
17,9,897.8,0.001
18,24,950.3,0.002
19,40,1107.7,0.004
20,74,1058.2,0.007
21,132,1094.8,0.013
22,196,1146.6,0.020


In [81]:
df['x2'] = df['x2'].clip(lower=25,upper=35)

In [82]:
for feature in [f'x{i}' for i in range(1, 11)]:
    df['strat'] = df[feature]
    stratified_var, min_part = calculate_strat_stat(df)
    print(f'{feature:<5} var={stratified_var:0.0f}, min_part={min_part*100:0.2f}%')

x1    var=66234, min_part=0.02%
x2    var=51996, min_part=6.69%
x3    var=64391, min_part=0.01%
x4    var=66054, min_part=32.61%
x5    var=66022, min_part=40.74%
x6    var=65714, min_part=0.01%
x7    var=48055, min_part=0.01%
x8    var=64859, min_part=34.95%
x9    var=66051, min_part=39.70%
x10   var=63755, min_part=13.62%


#### Strategy 3. Cross strat 

In [83]:
df['strat'] = [' '.join(values) for values in df[['x2', 'x10']].astype(str).values]
stratified_var, min_part = calculate_strat_stat(df)
print(f'var={stratified_var:0.0f}, min_part={min_part*100:0.2f}%')

var=49650, min_part=0.82%


In [84]:
df['strat']

0       30 1
1       27 2
2       29 0
3       34 1
4       26 2
        ... 
9995    31 3
9996    27 0
9997    28 2
9998    26 2
9999    26 2
Name: strat, Length: 10000, dtype: object

### ML approach

In [87]:
from lightgbm import LGBMRegressor

df_train = df.iloc[:len(df) // 2].copy()
df_test = df.iloc[len(df) // 2:].copy()

model = LGBMRegressor(num_leaves=3)
feature_names = [f'x{i}' for i in range(1, 11)]
model.fit(df_train[feature_names].values, df_train['y'].values)
predict_test = model.predict(df_test[feature_names].values)

n_strat = 10
quantiles = np.quantile(predict_test, np.linspace(0, 1 - 1 / n_strat, n_strat))
df_test['strat'] = [np.sum(predict >= quantiles) for predict in predict_test]

# var=42289, min_part=9.98%

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000238 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 744
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 10
[LightGBM] [Info] Start training from score 1370.817800


/Users/zabelinainessa/Jupyter_project/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [88]:
df_test['strat']

5000     9
5001     7
5002     4
5003     6
5004    10
        ..
9995    10
9996     6
9997     9
9998     2
9999     3
Name: strat, Length: 5000, dtype: int64

In [90]:
stratified_var, min_part = calculate_strat_stat(df_test)
print(f'var={stratified_var:0.0f}, min_part={min_part*100:0.2f}%')

var=42712, min_part=9.98%


### Task 2. Function of stratified distribution of clients by groups


In [111]:
import numpy as np


def split_stratified(strats):
    """Распределяет объекты по группам (контрольная и экспериментальная).

    :param strats (np.array): массив с разбиением на страты.
    :return groups (np.array): массив из 0 и 1,
        0 - контрольная группа, 1 - экспериментальная.
    """
    split_array = np.zeros(strats.size, dtype=int)
    
    unique_strats, counts = np.unique(strats, return_counts=True)
    
    for strat, count in zip(unique_strats, counts):        
        indices = np.where(strats == strat)[0]
        split_array[indices[:count//2]] = 1
    
    return split_array

In [113]:
df = pd.DataFrame({'strat': [1, 2, 2, 2, 1, 1, 1, 3, 3]})
df['group'] = split_stratified(df['strat'].values)
# df = pd.DataFrame({
#   'strat': [1, 2, 2, 2, 1, 1, 1, 3, 3],
#   'group': [1, 0, 0, 1, 0, 0, 1, 0, 1]
# })
df

,strat,group
0,1,1
1,2,1
2,2,0
3,2,0
4,1,1
5,1,0
6,1,0
7,3,1
8,3,0


### Task 3. Function using poststratification

#### Notes
----------------
np.var(**ddof=0** - default) - “population standard deviation” \
np.var(**ddof=1**) - “sample standard deviation”

pd.DataFrame.var(**ddof=1** - default)

In [215]:
import numpy as np
import pandas as pd
from scipy import stats


def get_ttest_strat_pvalue(metrics_strat_a_group, metrics_strat_b_group):
    """Применяет постстратификацию, возвращает pvalue.

    Веса страт считаем по данным обеих групп.
    Предполагаем, что эксперимент проводится на всей популяции.
    Веса страт нужно считать по данным всей популяции.

    :param metrics_strat_a_group (np.ndarray): значения метрик и страт группы A.
        shape = (n, 2), первый столбец - метрики, второй столбец - страты.
    :param metrics_strat_b_group (np.ndarray): значения метрик и страт группы B.
        shape = (n, 2), первый столбец - метрики, второй столбец - страты.
    :return (float): значение p-value
    """
    general_strat = np.vstack((metrics_strat_a_group, metrics_strat_b_group)).T[1]
    weights = np.unique(general_strat, return_counts=True)[1]/np.unique(general_strat, return_counts=True)[1].sum()
    unique_values = np.unique(general_strat)

    mean_control = np.array([metrics_strat_a_group[metrics_strat_a_group[:,1] == value, 0].mean() for value in unique_values])
    mean_pilot = np.array([metrics_strat_b_group[metrics_strat_b_group[:,1] == value, 0].mean() for value in unique_values])

    mean_strat_control = (mean_control*weights).sum()
    mean_strat_pilot= (mean_pilot*weights).sum()

    var_control = np.array([metrics_strat_a_group[metrics_strat_a_group[:,1] == value, 0].var(ddof=1) for value in unique_values])
    var_pilot = np.array([metrics_strat_b_group[metrics_strat_b_group[:,1] == value, 0].var(ddof=1) for value in unique_values])
    
    var_strat_control = (var_control*weights).sum()
    var_strat_pilot = (var_pilot*weights).sum()
    
    delta_mean_strat = mean_strat_pilot - mean_strat_control
    std_mean_strat = (var_strat_pilot / len(metrics_strat_a_group) + var_strat_control / len(metrics_strat_b_group)) ** 0.5
    t = delta_mean_strat / std_mean_strat
    pvalue = (1 - stats.norm.cdf(np.abs(t))) * 2
    return pvalue

In [219]:
metrics_strat_a_group = np.array([
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
]).T
metrics_strat_b_group = np.array([
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
]).T
pvalue = get_ttest_strat_pvalue(metrics_strat_a_group, metrics_strat_b_group)
pvalue
# pvalue = 0.037056

np.float64(0.037056218564119)